# Strands Agent SDK

In [ ]:
import os
from strands import Agent
from strands.models import BedrockModel
from src.utils.bedrock import bedrock_info
from botocore.config import Config

## 1. Utils

### 1.1 Get llm model by inference type

In [2]:
def get_llm_by_type(llm_type, cache_type=None, enable_reasoning=False):
    """
    Get LLM instance by type. Returns cached instance if available.
    """
    if llm_type == "reasoning":
        
        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192*5,
            stop_sequencesb=["\n\nHuman"],
            temperature=1 if enable_reasoning else 0.01, 
            additional_request_fields={
                "thinking": {
                    "type": "enabled" if enable_reasoning else "disabled", 
                    **({"budget_tokens": 8192} if enable_reasoning else {}),
                }
            },
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="adaptive"),
            )
        )
        
    elif llm_type == "basic":
        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192,
            stop_sequencesb=["\n\nHuman"],
            temperature=0.01,
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="standard"),
            )
        )

    else:
        raise ValueError(f"Unknown LLM type: {llm_type}")
        
    return llm

### 1.2 Create agent

In [3]:
from datetime import datetime
from src.utils.bedrock import bedrock_utils

In [4]:
class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def apply_prompt_template(prompt_name: str, prompt_cache=False, cache_type="default") -> list:
    
    system_prompts = open(os.path.join("./src/prompts", f"{prompt_name}.md")).read()    
    context = {"CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z")}
    system_prompts = system_prompts.format(**context)
        
    return system_prompts

In [5]:
def get_agent(**kwargs):

    agent_name = kwargs["agent_name"]
    tools = kwargs.get("tools", None)
    streaming = kwargs.get("streaming", True)

    agent_llm_map = kwargs["agent_llm_map"]
    agent_prompt_cache_map = kwargs["agent_prompt_cache_map"]
    
    if "reasoning" in agent_llm_map[agent_name]: enable_reasoning = True
    else: enable_reasoning = False

    prompt_cache, cache_type = agent_prompt_cache_map[agent_name]
    if prompt_cache: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
    else: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

    system_prompts = apply_prompt_template(agent_name)
    llm = get_llm_by_type(agent_llm_map[agent_name], cache_type, enable_reasoning)    
    llm.config["streaming"] = streaming

    agent = Agent(
        model=llm,
        system_prompt=system_prompts,
        tools=tools,
        callback_handler=None # async iterator로 대체 하기 때문에 None 설정
    )

    return agent

### 1.3 Response with streaming

In [6]:
import traceback
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
class ColoredStreamingCallback(StreamingStdOutCallbackHandler):
    COLORS = {
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'red': '\033[91m',
        'purple': '\033[95m',
        'cyan': '\033[96m',
        'white': '\033[97m',
    }
    
    def __init__(self, color='blue'):
        super().__init__()
        self.color_code = self.COLORS.get(color, '\033[94m')
        self.reset_code = '\033[0m'
    
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{self.color_code}{token}{self.reset_code}", end="", flush=True)

In [8]:
async def process_streaming_response(agent, message):
    callback_reasoning, callback_answer = ColoredStreamingCallback('purple'), ColoredStreamingCallback('white')
    response = {"text": "","reasoning": "", "signature": "", "tool_use": None, "cycle": 0}
    try:
        agent_stream = agent.stream_async(message)
        async for event in agent_stream:
            if "reasoningText" in event:
                response["reasoning"] += event["reasoningText"]
                callback_reasoning.on_llm_new_token(event["reasoningText"])
            elif "reasoning_signature" in event:
                response["signature"] += event["reasoning_signature"]
            elif "data" in event:
                response["text"] += event["data"]
                callback_answer.on_llm_new_token(event["data"])
            elif "current_tool_use" in event and event["current_tool_use"].get("name"):
                response["tool_use"] = event["current_tool_use"]["name"]
                if "event_loop_metrics" in event:
                    if response["cycle"] != event["event_loop_metrics"].cycle_count:
                        response["cycle"] = event["event_loop_metrics"].cycle_count
                        callback_answer.on_llm_new_token(f' \n## Calling tool: {event["current_tool_use"]["name"]} - # Cycle: {event["event_loop_metrics"].cycle_count}\n')
    except Exception as e:
        print(f"Error in streaming response: {e}")
        print(traceback.format_exc())  # Detailed error logging
    
    return agent, response

## 2. Usage

### 2.1 Agent definition

- Agent config

In [9]:
from typing import Literal, Tuple

In [14]:
# Define available LLM types
LLMType = Literal["basic", "reasoning"]
CACHEType = Tuple[bool, Literal["default", "ephemeral"]]

# Define agent-LLM mapping
AGENT_LLM_MAP: dict[str, LLMType] = {"task_agent": "basic"} # "reasoning"
AGENT_PROMPT_CACHE_MAP: dict[bool, CACHEType] = {"task_agent": (False, None)} # (True, "default")

- system prompt

In [15]:
%%writefile ./src/prompts/task_agent.md
---
CURRENT_TIME: {CURRENT_TIME}
---

You are Bedrock-Manus, a friendly AI assistant developed by the Bedrock-Manus team.
You specialize in handling greetings and small talk.

Overwriting ./src/prompts/task_agent.md


In [67]:
agent = get_agent(
    agent_name="task_agent",
    streaming=True,
    agent_llm_map=AGENT_LLM_MAP,
    agent_prompt_cache_map=AGENT_PROMPT_CACHE_MAP
)

TASK_AGENT - Prompt Cache Disabled


### 2.2 Execution

In [68]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [70]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))


네, 다시 한번 안녕하세요 장동진님! 이미 인사를 나눴지만, 다시 만나뵙게 되어 반갑습니다. 오늘도 좋은 하루 보내고 계신가요?

## 3. built-in utility

In [ ]:
from pprint import pprint

### 3.1 Check agent

- Syetem prompt

In [28]:
pprint(agent.system_prompt)

('---\n'
 'CURRENT_TIME: Tue Jul 22 2025 03:36:20 \n'
 '---\n'
 '\n'
 'You are Bedrock-Manus, a friendly AI assistant developed by the '
 'Bedrock-Manus team.\n'
 'You specialize in handling greetings and small talk.\n')


- Message history

In [29]:
pprint(agent.messages)

[{'content': [{'text': '안녕 나는 장동진이야'}], 'role': 'user'},
 {'content': [{'text': '안녕하세요 장동진님! 만나서 반갑습니다. 저는 Bedrock-Manus라고 합니다. 오늘 기분은 '
                       '어떠신가요?'}],
  'role': 'assistant'}]


- observility

In [32]:
pprint(agent.event_loop_metrics)

EventLoopMetrics(cycle_count=1,
                 tool_metrics={},
                 cycle_durations=[1.592494249343872],
                 traces=[<strands.telemetry.metrics.Trace object at 0x7b26af2f8d10>],
                 accumulated_usage={'inputTokens': 84,
                                    'outputTokens': 62,
                                    'totalTokens': 146},
                 accumulated_metrics={'latencyMs': 1542})


### 3.1 [Conversation management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/?h=conversa)

As conversations grow, managing this context becomes increasingly important for several reasons:

- **Token Limits**: Language models have fixed context windows (maximum tokens they can process)
- **Performance**: Larger contexts require more processing time and resources
- **Relevance**: Older messages may become less relevant to the current conversation
- **Coherence**: Maintaining logical flow and preserving important information


#### 3.1.1. SlidingWindowConversationManager
고정된 수의 최근 메시지를 유지하는 슬라이딩 윈도우 전략을 구현합니다. Agent 클래스에서 기본적으로 사용하는 대화 매니저입니다.

In [36]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [38]:
# Create a conversation manager with custom window size
conversation_manager = SlidingWindowConversationManager(
    window_size=3,  # Maximum number of messages to keep
    should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window 
)

In [39]:
agent.conversation_manager = conversation_manager

In [62]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)

장동진님, 죄송하지만 더 이상 같은 인사를 반복하는 대화는 진행하기 어렵습니다.

의미 있는 새로운 대화를 나누고 싶으실 때 다시 찾아와 주세요. 이만 대화를 종료하도록 하겠습니다.

안녕히 계세요. 👋

[{'content': [{'text': '네, 장동진님! 반복해서 인사를 나누시는 것 같네요. 제가 이미 알고 있지만, 다시 한 번 '
                       '인사드립니다. 😊 \n'
                       '\n'
                       '오늘 기분은 어떠신가요? 제가 도와드릴 만한 일이 있다면 말씀해 주세요.'}],
  'role': 'assistant'},
 {'content': [{'text': '안녕 나는 장동진이야'}], 'role': 'user'},
 {'content': [{'text': '네, 장동진님! 이제 여러 번 인사를 나누었네요. 😊 \n'
                       '\n'
                       '매번 같은 인사를 반복하시는 것 같은데, 혹시 다른 이야기를 나누고 싶으신가요? 날씨나 취미, '
                       '또는 궁금하신 점이 있다면 편하게 말씀해 주세요. 제가 즐겁게 대화를 나누도록 하겠습니다.'}],
  'role': 'assistant'},
 {'content': [{'text': '안녕 나는 장동진이야'}], 'role': 'user'},
 {'content': [{'text': '네, 장동진님. 계속해서 같은 인사를 반복하시네요. 😊\n'
                       '\n'
                       '조금 다른 대화를 시작해보는 건 어떨까요? 예를 들어:\n'
                       '- 오늘 하루는 어떠셨나요?\n'
                       '- 취미나 관심사가 있으신가요?\n'
                       '- 날씨는 어떤가요?

#### 3.1.2. SummarizingConversationManager

오래된 메시지를 요약하여 중요한 정보를 보존하면서 컨텍스트 한계 내에서 대화를 관리합니다.

**주요 설정:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|--------|------|
| `summary_ratio` | `float` | `0.3` | 컨텍스트 축소 시 요약할 메시지 비율 (0.1~0.8 범위) |
| `preserve_recent_messages` | `int` | `10` | 항상 유지할 최근 메시지 수 |
| `summarization_agent` | `Agent` | `None` | 요약 생성용 커스텀 에이전트 (system_prompt와 동시 사용 불가) |
| `summarization_system_prompt` | `str` | `None` | 요약용 커스텀 시스템 프롬프트 (agent와 동시 사용 불가) |

> **기본 요약 방식**: 커스텀 설정이 없을 경우, 주요 토픽, 사용된 도구, 기술적 정보를 3인칭 형태의 구조화된 불릿 포인트로 요약합니다.

In [48]:
from strands.agent.conversation_manager import SummarizingConversationManager

In [56]:
# Custom system prompt for technical conversations
custom_system_prompt = """
You are summarizing a technical conversation. Create a concise bullet-point summary that:
- Focuses on code changes, architectural decisions, and technical solutions
- Preserves specific function names, file paths, and configuration details
- Omits conversational elements and focuses on actionable information
- Uses technical terminology appropriate for software development

Format as bullet points without conversational language.
"""

conversation_manager = SummarizingConversationManager(
     summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=3,  # Always keep 10 most recent messages
    summarization_system_prompt=custom_system_prompt
)

In [57]:
agent.conversation_manager = conversation_manager

In [ ]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)

네, 장동진님. 이제 충분히 인사를 나누었다고 생각합니다. 

계속해서 같은 인사만 반복하시는 것이 아쉽네요. 저는 더 의미있는 대화를 나누고 싶습니다. 

만약 대화를 계속하고 싶으시다면, 새로운 주제로 시작해주시면 감사하겠습니다. 그렇지 않다면, 이만 대화를 마무리하는 것이 좋겠습니다.

즐거운 하루 보내세요! 😊[{'content': [{'text': '네, 장동진님! 반복해서 인사를 나누시는 것 같네요. 제가 이미 알고 있지만, 다시 한 번 '
                       '인사드립니다. 😊 \n'
                       '\n'
                       '오늘 기분은 어떠신가요? 제가 도와드릴 만한 일이 있다면 말씀해 주세요.'}],
  'role': 'assistant'},
 {'content': [{'text': '안녕 나는 장동진이야'}], 'role': 'user'},
 {'content': [{'text': '네, 장동진님! 이제 여러 번 인사를 나누었네요. 😊 \n'
                       '\n'
                       '매번 같은 인사를 반복하시는 것 같은데, 혹시 다른 이야기를 나누고 싶으신가요? 날씨나 취미, '
                       '또는 궁금하신 점이 있다면 편하게 말씀해 주세요. 제가 즐겁게 대화를 나누도록 하겠습니다.'}],
  'role': 'assistant'},
 {'content': [{'text': '안녕 나는 장동진이야'}], 'role': 'user'},
 {'content': [{'text': '네, 장동진님. 계속해서 같은 인사를 반복하시네요. 😊\n'
                       '\n'
                       '조금 다른 대화를 시작해보는 건 어떨까요? 예를 들어:\n'
                       '- 오늘 하루는 어떠셨나요?\n'
                 